In [ ]:
!nvidia-smi 

In [ ]:
%%capture
!pip install datasets
!pip install transfomers
!pip install wandb
!pip install sentence-transformers
!pip install trl

In [ ]:
!wandb login

In [ ]:
import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd


from datasets import load_dataset

from transformers import AutoTokenizer
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

# old code, using for init models
config = {
    "model_name": "gpt2-large",

}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe_device = 0 if torch.cuda.is_available() else -1

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer


gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained(config['model_name'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model_name'])#,padding_side='left')

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained(config['model_name'])
gpt2_model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(config['model_name'])

In [ ]:
gpt2_model.to(device)
gpt2_model_ref.to(device)
None

In [ ]:
#gpt2_model_ref

# About data
 - I using cos_e data, its contains a QA pairs with extractive explanation, so its looks like:
 - Its close to common sense reasoning tasks, and also could be solved in ZS style with bigger models
___
Question: When you drive and see someone walking across the street what do we do?
Answer: see someone walking
___


In [ ]:
from datasets import load_dataset

dataset = load_dataset("neulab/conala",'curated')
qs = dataset['train']['rewritten_intent']
ans = dataset['train']['snippet']

In [ ]:
import pandas as pd


data_qa = []
for question, answer in zip(qs ,ans):
    data_qa+=[f"Question: {question} Answer: {answer} <|endoftext|>"]

In [ ]:
print(data_qa[0])

In [ ]:

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('paraphrase-mpnet-base-v2')


In [ ]:
corpus_embeddings = embedder.encode(data_qa, convert_to_tensor=True)

In [ ]:

import numpy as np

def _ans(text,top_k = 1):
    """
    - returns a closest questions - answer pairs to prompted
    """
    queries = [text.split('answer')[0]]

    ans =[]
    
    top_k = top_k
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)

        
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)

        
        return [data_qa[top_results[1][0]]]# data_qa[top_results[1][1]], data_qa[top_results[1][2]], data_qa[top_results[1][3]], data_qa[top_results[1][4]] ]
       

    

In [ ]:
_ans('Question: Jay & Gloria were hosting a 4th of July party at their house.  Jay invited 22 people and Gloria invited 36.  They wanted to buy small American flags for everyone.  The craft store was having a sale on small flags, 5 flags for $1.00.  If they wanted all of their guests to have a flag and they also wanted 1 flag each, how much would they spend at the craft store?')

In [ ]:
from torch.utils.data import Dataset
from tqdm.contrib import tzip
from datasets import load_dataset


class QADataset(Dataset):
    def __init__(self,tokenizer):
      
      dataset = load_dataset("neulab/conala",'curated')
      qs = dataset['train']['rewritten_intent']
      ans = dataset['train']['snippet']

      self.tokenized = []
      self.t_q = []
      self.t_a = []

      for a, q in tzip(ans,qs):

        pr = f'Question: {q} Answer: {a} {tokenizer.eos_token}'

        enc = self._encode(text=pr, tokenizer=tokenizer)
        self.tokenized +=[enc]
        
      
      

      

    def __len__(self):
        return len(self.tokenized)

    def __getitem__(self, item):
        return (self.tokenized[item])

    def _encode(self, text, tokenizer):
        
        encoded_sample = tokenizer.encode(text,  padding='max_length', max_length=512, return_tensors='pt')

        return encoded_sample

In [ ]:
from torch.utils.data import DataLoader
data = QADataset(tokenizer=tokenizer)

In [ ]:
train_dataloader = DataLoader(
        data, shuffle=True, batch_size=10,  drop_last=True)


In [ ]:
len(train_dataloader)

# A SFT function
- We gonna make a finetune every epoch, simmilar to OPENAI paper

In [ ]:
import torch
from torch import optim


class AdamWEMA(optim.Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=1e-2, amsgrad=False, ema_decay=0.999,
                 ema_power=1., param_names=()):
        """AdamW that saves EMA versions of the parameters."""
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if not 0.0 <= ema_decay <= 1.0:
            raise ValueError("Invalid ema_decay value: {}".format(ema_decay))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad, ema_decay=ema_decay,
                        ema_power=ema_power, param_names=param_names)
        super().__init__(params, defaults)

    def __setstate__(self, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.
        Args:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            params_with_grad = []
            grads = []
            exp_avgs = []
            exp_avg_sqs = []
            ema_params_with_grad = []
            state_sums = []
            max_exp_avg_sqs = []
            state_steps = []
            amsgrad = group['amsgrad']
            beta1, beta2 = group['betas']
            ema_decay = group['ema_decay']
            ema_power = group['ema_power']

            for p in group['params']:
                if p.grad is None:
                    continue
                params_with_grad.append(p)
                if p.grad.is_sparse:
                    raise RuntimeError('AdamW does not support sparse gradients')
                grads.append(p.grad)

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    # Exponential moving average of parameter values
                    state['param_exp_avg'] = p.detach().float().clone()

                exp_avgs.append(state['exp_avg'])
                exp_avg_sqs.append(state['exp_avg_sq'])
                ema_params_with_grad.append(state['param_exp_avg'])

                if amsgrad:
                    max_exp_avg_sqs.append(state['max_exp_avg_sq'])

                # update the steps for each param group update
                state['step'] += 1
                # record the step after step update
                state_steps.append(state['step'])

            optim._functional.adamw(params_with_grad,
                    grads,
                    exp_avgs,
                    exp_avg_sqs,
                    max_exp_avg_sqs,
                    state_steps,
                    amsgrad=amsgrad,
                    beta1=beta1,
                    beta2=beta2,
                    lr=group['lr'],
                    weight_decay=group['weight_decay'],
                    eps=group['eps'],
                    maximize=False)

            cur_ema_decay = min(ema_decay, 1 - state['step'] ** -ema_power)
            for param, ema_param in zip(params_with_grad, ema_params_with_grad):
                ema_param.mul_(cur_ema_decay).add_(param.float(), alpha=1 - cur_ema_decay)

        return loss

In [ ]:
wandb.init()

In [ ]:
def sft(model, train_dataloader):
    from transformers import  AdamW, get_linear_schedule_with_warmup


    # also i reinint optimizer and scheduler, idn its works better 
    optimizer = AdamW(gpt2_model.parameters(), lr = 2e-5)
    scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=100, num_training_steps=len(train_dataloader)
            )
    model.train()
    for batch in tqdm(train_dataloader):

        t = batch[0].to(device)

        loss = gpt2_model.forward(input_ids = t, labels=t)[1]

        #wandb.log({"SFT loss":  loss})

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        loss.backward()
        optimizer.step()
        scheduler.step()
    model.eval()
    del optimizer
    torch.cuda.empty_cache()
        



In [ ]:
!mkdir ckpts5_sd

In [ ]:
from tqdm import tqdm
for i in tqdm(range(10)):
    
    sft(gpt2_model, train_dataloader)
    
    gpt2_model.save_pretrained(f'ckpts5_sd/sft_{i}')
    

In [ ]:
emb1 = embedder.encode('a1')
emb2 = embedder.encode('a2')

In [ ]:
torch.tensor(emb1).size()

In [ ]:
torch.tensor(emb2).size()

In [ ]:
mse_loss = torch.nn.functional.mse_loss(torch.tensor(emb1), torch.tensor(emb2))

In [ ]:
mse_loss

In [ ]:
def detect_q(a1, a2):
    """
    - checking its not a question
    """
    emb1 = embedder.encode(a1)
    emb2 = embedder.encode(a2)
    cos_sim = util.cos_sim(emb2,emb1)
    return float(cos_sim)

def detect_intent(a1, a2):
    """
    measure how close ground true answer to predicted
    """
    a1 = a1
    a2 = a2
    emb1 = embedder.encode(a1)
    emb2 = embedder.encode(a2)
    cos_sim = util.cos_sim( emb2, emb1) #torch.nn.functional.mse_loss(torch.tensor(emb1), torch.tensor(emb2))
    cos_sim = (cos_sim + 1) / 2
    return float(cos_sim)

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

tokenizer_b = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
#embedder = SentenceTransformer('paraphrase-mpnet-base-v2')

def reward(qa):
    ans = _ans(qa)
    q = qa.split('Answer:')[0]
    answer = qa.split('Answer:')[1]
    
    if '\n' in answer:
        return 0
    max_length = 150
    if len(tokenizer.encode(answer)) > max_length:
        return -1 * (len(tokenizer.encode(answer)) - max_length)  # Subtract points for every 50 tokens beyond the max_length
    emb1 = embedder.encode(q)
    emb2 = embedder.encode(answer)
    cos_sim = util.cos_sim(emb2, emb1).item()
    
    eucl_dist = euclidean_distances(emb2.reshape(1,-1), emb1.reshape(1,-1))[0][0]
    
    cos_dist = cosine_distances(emb2.reshape(1,-1), emb1.reshape(1,-1))[0][0]
    
    qa_encoding = tokenizer_b(qa, return_tensors="pt")
    with torch.no_grad():
        qa_emb = model(**qa_encoding).last_hidden_state.mean(dim=1).squeeze(0)  # Compute mean pooling of the BERT embeddings
    
    cos_sim_qa = util.cos_sim(emb2, qa_emb.unsqueeze(0)).item()  # Compute cosine similarity between the predicted answer and the context encoding
    score = (cos_sim + 1) / 2
    score += 1 - eucl_dist  # Add Euclidean distance score
    score += 1 - cos_dist  # Add Cosine distance score
    score += (cos_sim_qa + 1) / 2  # Add contextual similarity score
    score /= 4  # Average the scores
    if ans:
        gts = ans
        scores = []
        for gt in gts:
            s = detect_intent(gt, answer)
            scores+=[s]
        score = max(scores)
        if score > 0.7:
            score *= 2  # Double the score if it's high enough
        return score
    else:
        return score


## cheking PPO hacking reward cased

In [ ]:
qa = """Question: Concatenate elements of a list 'x' of multiple integers to a single integer Answer: sum(d * 10 ** i for i, d in enumerate(x[::-1])) <|endoftext|>"""

In [ ]:
reward(qa)

In [ ]:
q = qa.split('Answer:')[0]
answer = qa.split('Answer:')[1]

In [ ]:
answer

In [ ]:
q

In [ ]:
detect_intent(q, answer)

In [ ]:
answer

In [ ]:
reward('Question: how are glacier caves formed? Answer: how are glacier caves formed? ')


In [ ]:
reward("""Question: What are you a part of if you hear a bugle in the mornings?? Answer:What are you a part of if you hear a bugle in the mornings?

I am not a part of if you hear a bugle in the mornings.

What are you a part of if you hear a bugle in the mornings?

I am""")

In [ ]:
output_min_length = 20
output_max_length = 200
output_length_sampler = LengthSampler(output_min_length, output_max_length)


In [ ]:
ds = dataset['train']

In [ ]:
tokenizer.padding_side = 'left'

In [ ]:
def tokenize(sample):
    sample["tokens"] = tokenizer.encode(f"Question: { sample['rewritten_intent'] } Answer:",padding='max_length')
    #'Question: {q} Answer: {a} {tokenizer.eos_token}
    sample["query"] = f" {tokenizer.decode(sample['tokens'])}"
    return sample

ds = ds.map(tokenize, batched=False)

In [ ]:
class QDataset(Dataset):
    def __init__(self,tokenizer):
      
      dataset = load_dataset("neulab/conala",'curated')
      qs = dataset['train']['rewritten_intent']
     

      self.tokenized = []
      self.t_q = []
      self.t_a = []

      for a, q in tzip(ans,qs):

        pr = f'Question: {q} Answer:'

        enc = self._encode(text=pr, tokenizer=tokenizer)
        self.tokenized +=[enc]
        
      
      

      

    def __len__(self):
        return len(self.tokenized)

    def __getitem__(self, item):
        return (self.tokenized[item])

    def _encode(self, text, tokenizer):
        
        encoded_sample = tokenizer.encode(text,  padding='max_length', max_length=128, return_tensors='pt')

        return encoded_sample

In [ ]:
da = QDataset(tokenizer)

In [ ]:
gen_kwargs = {
    "min_length": output_min_length,
    "max_new_tokens": 100,
    "top_k": 50,
    "top_p": 0.7,
    "do_sample": False,  # Use beam search instead of sampling
    "no_repeat_ngram_size": 2,
    "pad_token_id": tokenizer.eos_token_id,
    "use_cache": True,
    #"num_beams": 1,  # Use beam search with 5 beams
    "repetition_penalty": 1.5,  # Add a penalty for repetition
    "length_penalty": 0.8  # Add a penalty for long responses
}


In [ ]:
config = PPOConfig(
    model_name="gpt2",
    learning_rate=1.41e-6,
    log_with="wandb",
    vf_coef = 0.0001,
    ppo_epochs = 4,
    batch_size = 64
)

In [ ]:
config.batch_size

In [ ]:
# making ckpts di
!mkdir ckpts5

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

from torch.utils.data import DataLoader
data = QDataset(tokenizer=tokenizer)
dataloader =  torch.utils.data.DataLoader(data, batch_size=config.batch_size)

In [ ]:
config.batch_size

In [ ]:
ppo_trainer = PPOTrainer(config, gpt2_model, gpt2_model_ref, tokenizer, dataset=data, data_collator=collator)

In [ ]:
ppo_trainer

In [ ]:
def to_log(li):
    q = []
    r = []
    for i in li:
        q +=[i.split('Answer:')[0]]
        r +=[i.split('Answer:')[1]]
    data = {'query':q,'response':r}
    return data

In [ ]:
import gc
from tqdm import tqdm
epoch = 0
glob_epo = 5
for epo in tqdm(range(glob_epo)):
    
    for batch in tqdm(dataloader):
        try:
            epoch+=1
            logs = dict()
            batch = batch.to('cuda')
            gen = gpt2_model.generate(batch.squeeze(1), **gen_kwargs)
            a = tokenizer.batch_decode(gen)
            rewards_l = []
            rsw = []
            i = 0
            for text in a:
                text = text.replace('<|endoftext|>','')
                try:

                    x = reward(text)
                except:
                    x = 0
                    print(text)

                i+=1
                #print(x, text)
                rsw += [x]   
                rewards_l += [torch.tensor(x, dtype=torch.float32)]

            rewards = rewards_l
            query_tensors = batch.squeeze(1).cpu()
            response_tensors = gen.squeeze(1).cpu()

            del gen
            del batch
            gc.collect()
            torch.cuda.empty_cache()

            query_tensors = [tensor for tensor in query_tensors]
            response_tensors = [tensor for tensor in response_tensors]


            stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
            ppo_trainer.log_stats(stats,  to_log(a), rewards)

            logs['env/reward_mean'] = np.array(rewards).mean()
            logs['env/reward_std'] = np.array(rewards).std()
            logs['env/reward_dist'] = rewards
            wandb.log(logs)

            gpt2_model.save_pretrained(f'ckpts5/ppo_{glob_epo}.{epoch}')
        except Exception as e:
            print(e)


    sft(gpt2_model, train_dataloader)
    
    gpt2_model.save_pretrained(f'ckpts5/sft_{glob_epo}')
    

In [ ]:
sft(gpt2_model, train_dataloader)
    
gpt2_model.save_pretrained(f'ckpts5/sft_{glob_epo}')

In [ ]:
np.array(rewards).mean()

In [ ]:
ppo_trainer.log_stats(stats,  to_log(a), rewards)

In [ ]:
query_tensors

In [ ]:
_response_tensors = []
ll = response_tensors.tolist()
for l in ll:
    _response_tensors+=torch.Tensor(l)


In [ ]:
list_of_tensors = [tensor for tensor in response_tensors]

In [ ]:
list_of_tensors

In [ ]:
text

In [ ]:
batch.size()

In [ ]:
a

In [ ]:
query_tensors[2]

In [ ]:
from tqdm.contrib import tzip

total_ppo_epochs = int(np.ceil(args["steps"]/config.batch_size))
print(total_ppo_epochs)

for epoch, batch in tzip(range(total_ppo_epochs), iter(dataloader)):
   
    
    logs, timing = dict(), dict()
   
    query_tensors = [torch.tensor(t).long().to(device) for t in batch["tokens"]]
    
    
    
    response_tensors = []
    for i in range(config.batch_size):
        try:
            response = gpt2_model.generate(query_tensors[i].unsqueeze(dim=0),
                                       **gen_kwargs)
            response_tensors.append(response.squeeze())
        except Exception as e:
            print(e)
    batch['response'] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    texts = [r for q,r in zip(batch['query'], batch['response'])]
    rewards_l = []
    rsw = []
    i = 0
    for text in texts:
        x = reward(text)
        i+=1
         
        rsw += [x]   
        rewards_l += [torch.tensor(x)]

    rewards = rewards_l
    
    
    
    #### Run PPO step 
    
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
        
    rewards = np.array(rsw)
    table_rows = [list(r) for r in zip(batch['query'], batch['response'], rewards)]
    logs.update({'game_log': wandb.Table(columns=['query', 'response', 'reward'], rows=table_rows)})
    
    logs['env/reward_mean'] = rewards.mean()
    logs['env/reward_std'] = rewards.std()
    logs['env/reward_dist'] = rewards
    wandb.log(logs)
    
    gpt2_model.save_pretrained(f'ckpts3/ppo_{epoch}')
    
    sft(gpt2_model, train_dataloader)
    
    gpt2_model.save_pretrained(f'ckpts3/sft_{epoch}')
    
    
        
    
    
   

    


In [ ]:
gpt2_model = gpt2_model.from_pretrained('ckpts1/sft_28')

In [ ]:
gpt2_model.to(device)
None

In [ ]:
bs = 16
game_data = dict()
ds.set_format("pandas")
df_batch = ds[:].sample(bs)
game_data['query'] = df_batch['query'].tolist()
query_tensors = df_batch['tokens'].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    
    output = gpt2_model_ref.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device),
                                      **gen_kwargs).squeeze()
    response_tensors_ref.append(output)
    output = gpt2_model.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device),
                                 **gen_kwargs).squeeze()
    response_tensors.append(output)

#### decode responses
game_data['response (before)'] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data['response (after)'] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]



In [ ]:
#### sentiment analysis of query/response pairs before/after
texts = [ r for q,r in zip(game_data['query'], game_data['response (before)'])]


In [ ]:
rewards = []
for text in texts:
        x = reward(text)
           
        rewards += [x]

In [ ]:
game_data['rewards (before)'] = rewards

In [ ]:
texts = [r for q,r in zip(game_data['query'], game_data['response (after)'])]


In [ ]:
#### sentiment analysis of query/response pairs before/after
rewards_ = []
texts = [ r for q,r in zip(game_data['query'], game_data['response (after)'])]
for text in texts:
        x = reward(text)
           
        rewards_ += [x]
        

In [ ]:
game_data['rewards (after)'] = rewards_

In [ ]:
df_results = pd.DataFrame(game_data)
df_results

In [ ]:
df_results['rewards (before)'].mean()

In [ ]:
df_results['rewards (after)'].mean()

In [ ]:
df_results.to_csv('tuned.csv')